In [1]:
import pandas as pd
import numpy as np

# Sample

In [2]:
dt_index = pd.date_range('2000-01-01', freq = 'T', periods = 4)
ss = pd.Series([0.0, None, 2.0, 3.0], index = dt_index)
df = pd.DataFrame({'s': ss})

df

,s
2000-01-01 00:00:00,0.0
2000-01-01 00:01:00,NaN
2000-01-01 00:02:00,2.0
2000-01-01 00:03:00,3.0


In [3]:
df.asfreq(freq = '2min')

,s
2000-01-01 00:00:00,0.0
2000-01-01 00:02:00,2.0


# Resample

In [4]:
dt_index = pd.date_range('2018-08-03', periods = 12, freq = 'T')

ss = pd.Series(	np.arange(len(dt_index)), 
				index = dt_index)

ss.loc['2018-08-03 00:02:00':'2018-08-03 00:03:00'] = 4 
ss.loc['2018-08-03 00:07:00':'2018-08-03 00:08:00'] = 6

ss

2018-08-03 00:00:00     0
2018-08-03 00:01:00     1
2018-08-03 00:02:00     4
2018-08-03 00:03:00     4
2018-08-03 00:04:00     4
2018-08-03 00:05:00     5
2018-08-03 00:06:00     6
2018-08-03 00:07:00     6
2018-08-03 00:08:00     6
2018-08-03 00:09:00     9
2018-08-03 00:10:00    10
2018-08-03 00:11:00    11
Freq: T, dtype: int64

In [5]:
'''
label: 指定resample後的結果其index要使用sample區間的最左邊或最右邊的index
closed: 指定閉區間

第一個sample區間：[2018-08-03 00:00:00, 2018-08-03 00:05:00)
第二個sample區間：[2018-08-03 00:05:00, 2018-08-03 00:10:00)
第三個sample區間：[2018-08-03 00:10:00, 2018-08-03 00:15:00)
'''
rtn = ss.resample(rule = '5min', label = 'left', closed = 'left')

print(rtn.apply(lambda x: x.value_counts()))
print('----')
rtn.apply(lambda x: x.value_counts().index[x.value_counts().argmax()])

2018-08-03 00:00:00  4     3
                     0     1
                     1     1
2018-08-03 00:05:00  6     3
                     5     1
                     9     1
2018-08-03 00:10:00  10    1
                     11    1
dtype: int64
----


2018-08-03 00:00:00     4
2018-08-03 00:05:00     6
2018-08-03 00:10:00    10
Freq: 5T, dtype: int64

In [6]:
'''
第一個sample區間：[2018-08-03 00:00:00, 2018-08-03 00:05:00)
第二個sample區間：[2018-08-03 00:05:00, 2018-08-03 00:10:00)
第三個sample區間：[2018-08-03 00:10:00, 2018-08-03 00:15:00)
'''
rtn = ss.resample(rule = '5min', label = 'right', closed = 'left').sum()

rtn

2018-08-03 00:05:00    13
2018-08-03 00:10:00    32
2018-08-03 00:15:00    21
Freq: 5T, dtype: int64

In [5]:
'''
第零個sample區間：(2018-08-02 23:55:00, 2018-08-03 00:00:00]
第一個sample區間：(2018-08-03 00:00:00, 2018-08-03 00:05:00]
第二個sample區間：(2018-08-03 00:05:00, 2018-08-03 00:10:00]
第三個sample區間：(2018-08-03 00:10:00, 2018-08-03 00:15:00]
'''
rtn = ss.resample(rule = '5min', label = 'left', closed = 'right').sum()

rtn

2018-08-02 23:55:00     0
2018-08-03 00:00:00    15
2018-08-03 00:05:00    40
2018-08-03 00:10:00    11
Freq: 5T, dtype: int64

In [25]:
'''
第零個sample區間：(2018-08-02 23:55:00, 2018-08-03 00:00:00]
第一個sample區間：(2018-08-03 00:00:00, 2018-08-03 00:05:00]
第二個sample區間：(2018-08-03 00:05:00, 2018-08-03 00:10:00]
第三個sample區間：(2018-08-03 00:10:00, 2018-08-03 00:15:00]
'''
rtn = ss.resample(rule = '5min', label = 'right', closed = 'right').sum()

rtn

2018-08-03 00:00:00     0
2018-08-03 00:05:00    15
2018-08-03 00:10:00    40
2018-08-03 00:15:00    11
Freq: 5T, dtype: int64

In [30]:
'''
改以時間點當column
'''
df_sim = pd.DataFrame({'device_id':1, 'value':ss.resample(rule = '5min', label = 'right', closed = 'right').sum()})

df_sim.pivot_table(values= 'value', index = 'device_id', columns=df_sim.index)


,2018-08-03 00:00:00,2018-08-03 00:05:00,2018-08-03 00:10:00,2018-08-03 00:15:00
device_id,,,,
1,0,15,40,11


# 分組取樣

In [3]:
dt = pd.date_range('2018-08-03 00:00', periods = 10, freq = '1min')

df2 = pd.DataFrame({'data_time': dt.repeat(3),
					'site_id': np.array([1]*30),
					'device_id': np.tile(['a', 'b', 'c'], 10),
					'value': np.arange(30)})

df2.sort_values(by = ['device_id', 'data_time'])

,data_time,site_id,device_id,value
0,2018-08-03 00:00:00,1,a,0
3,2018-08-03 00:01:00,1,a,3
6,2018-08-03 00:02:00,1,a,6
9,2018-08-03 00:03:00,1,a,9
12,2018-08-03 00:04:00,1,a,12
15,2018-08-03 00:05:00,1,a,15
18,2018-08-03 00:06:00,1,a,18
21,2018-08-03 00:07:00,1,a,21
24,2018-08-03 00:08:00,1,a,24
27,2018-08-03 00:09:00,1,a,27


In [5]:
print(df2[df2['device_id'] == 'b'])
print('-'*56)

df_rtn = df2.groupby(['site_id', 'device_id', pd.Grouper(key = 'data_time', freq = '5min', closed = 'left', label = 'left')]).sum()

df_rtn.to_csv('tmp.csv')
df_rtn

             data_time  site_id device_id  value
1  2018-08-03 00:00:00        1         b      1
4  2018-08-03 00:01:00        1         b      4
7  2018-08-03 00:02:00        1         b      7
10 2018-08-03 00:03:00        1         b     10
13 2018-08-03 00:04:00        1         b     13
16 2018-08-03 00:05:00        1         b     16
19 2018-08-03 00:06:00        1         b     19
22 2018-08-03 00:07:00        1         b     22
25 2018-08-03 00:08:00        1         b     25
28 2018-08-03 00:09:00        1         b     28
--------------------------------------------------------


value
site_id device_id data_time                 
1       a         2018-08-03 00:00:00     30
                  2018-08-03 00:05:00    105
        b         2018-08-03 00:00:00     35
                  2018-08-03 00:05:00    110
        c         2018-08-03 00:00:00     40
                  2018-08-03 00:05:00    115